<a href="https://colab.research.google.com/github/Valentinasierra98/Proyecto_AL/blob/main/STREAMLIT_Proyecto_Giraldo_Sierra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROYECTO FINAL
## Curso: Aprendizaje Automático  
### Maestría en Inteligencia Artificial y Ciencia de Datos  

---

## Elaborado por:  
- **Jahir Alberto Giraldo (22500239)**  
- **Valentina Sierra Caña (22500212)**  

---


# Manual de Usuario - Aplicación Modelo de Predicción de Depósitos Bancarios

---

## Introducción

Esta aplicación web permite predecir si un cliente bancario aceptará una oferta de depósito a plazo fijo, utilizando un modelo de aprendizaje automático basado en Support Vector Classification (SVC). La app está diseñada para que equipos comerciales y de marketing puedan enfocar sus esfuerzos en clientes con mayor probabilidad de conversión.

---

## Requisitos Previos

- Tener acceso a Google Colab o cualquier entorno donde se pueda ejecutar Python.
- Contar con un archivo CSV con datos de clientes (solo para la opción de predicción por lote).
- Conexión a internet para descargar paquetes y ejecutar la app.

---

## Pasos para Ejecutar la Aplicación

### 1. Subir Archivo CSV en Google Colab

Para que la aplicación funcione correctamente, es necesario subir un archivo CSV con los datos que se usaron para entrenar el modelo. Este archivo debe contener las características de los clientes y debe llamarse exactamente bank.csv. Asegúrate de que el archivo tenga el formato adecuado y que incluya todas las columnas requeridas para que el modelo pueda procesar la información y realizar predicciones precisas.

### 2. Crear y Ejecutar la Aplicación

El código app.py contiene toda la lógica del modelo y la interfaz web con Streamlit. Para crear y ejecutar la app, sigue estos pasos:

El código crea automáticamente el archivo app.py.

Instala las librerías necesarias (streamlit, pyngrok).

# Uso de la Aplicación

Al abrir la app, verás el menú lateral con varias opciones:

### A. Información del Modelo
- **Qué es:**  
  Descripción del modelo SVM y sus parámetros.

- **Qué muestra:**  
  Variables más importantes y una vista previa de los datos de entrenamiento.

- **Para qué sirve:**  
  Entender cómo funciona el modelo y qué datos utiliza.

### B. Evaluación del Modelo
- Aquí puedes entrenar el modelo con los datos disponibles.

- Al entrenar, verás métricas clave como Accuracy, Precision, Recall y F1-score.

- También podrás observar:  
  - Matriz de confusión  
  - Curva ROC (evaluación visual del rendimiento)  
  - Reporte completo de clasificación

- Puedes descargar el reporte en formato CSV para análisis posterior.

### C. Predicción Individual
- Formulario para ingresar datos de un cliente específico.

- Campos como edad, trabajo, estado civil, balance, duración de llamada, etc.

- Al enviar, la app predice si el cliente aceptará el depósito y muestra la probabilidad.

- Incluye una barra de progreso visual de la probabilidad.

### D. Predicción por Lote
- Puedes subir un archivo CSV con varios registros de clientes.

- La app procesará el archivo y generará predicciones para todos los clientes.

- Muestra una tabla con los resultados y probabilidades.

- Permite descargar un archivo CSV con las predicciones.

In [ ]:
# Subir archivo CSV desde la PC en Google Colab
from google.colab import files
uploaded = files.upload()

# Verificar que se haya subido un archivo .csv
import os

csv_filename = None
for fname in uploaded.keys():
    if fname.lower().endswith(".csv"):
        csv_filename = fname
        print(f" Archivo CSV detectado: {csv_filename}")
        break

if not csv_filename:
    raise ValueError(" No se subió un archivo CSV válido.")

#STREAMLIT CREACIÓN DEL APP.PY

In [ ]:
# Crear archivo app.py con solo el modelo ganador SVM
app_code = '''
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import (classification_report, confusion_matrix,
                           roc_curve, auc, precision_recall_curve,
                           accuracy_score, recall_score, precision_score, f1_score)
from sklearn.calibration import calibration_curve
import io
import joblib
import time
from PIL import Image

# Configuración de la página
st.set_page_config(
    page_title="Bank Marketing Predictor Pro",
    page_icon="🏦",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Cargar logo (Pondremos un emoji)
def load_logo():
    # Esto es un placeholder - puedes reemplazarlo con Image.open("tu_logo.png")
    logo = Image.new('RGB', (100, 50), color=(73, 109, 137))
    return logo

logo = load_logo()

# Título de la aplicación con logo
st.image(logo, width=100)
st.title("🏦 Modelo de Predicción de Depósitos")
st.markdown("""
**Esta aplicación utiliza un modelo de clasificación SVC (Support Vector Classification)
optimizado para predecir si un cliente aceptará una oferta de depósito a plazo.
El modelo ha sido entrenado con datos reales de campañas de marketing bancario,
considerando variables como edad, ocupación, historial crediticio y comportamiento en campañas anteriores.
Al ingresar las características de un cliente, la aplicación estima la probabilidad de aceptación,
ayudando a los equipos comerciales y de marketing a tomar decisiones más informadas
y enfocar sus esfuerzos en clientes con mayor probabilidad de conversión.**
""")

# Cargar los datos
@st.cache_data
def load_data():
    df = pd.read_csv('bank.csv')
    df = df.drop_duplicates()
    return df

df = load_data()

# Preprocesamiento
def preprocess_data(df):
    # Codificación one-hot
    cat_cols = df.select_dtypes(include=['object']).columns
    df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

    # Eliminar variables con alta colinealidad
    cor_matrix = df_encoded.corr().abs()
    upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(bool))
    to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.8)]
    df_encoded = df_encoded.drop(columns=to_drop)

    # División de datos
    y = df_encoded['deposit_yes'].astype(int)
    X = df_encoded.drop('deposit_yes', axis=1)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Escalado
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test, scaler, X.columns

# Modelo ganador
def train_winner_model(X_train, y_train):
    model = SVC(
        C=0.71,
        kernel='poly',
        class_weight='balanced',
        probability=True,
        random_state=42
    )
    model.fit(X_train, y_train)
    return model

# Función para predecir nuevos datos
def predict_new_data(model, scaler, input_data, feature_names):
    # Preprocesar los datos de entrada
    input_df = pd.DataFrame([input_data])

    # One-hot encoding para las variables categóricas
    cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
    for col in cat_cols:
        if col in input_df.columns:
            input_df = pd.get_dummies(input_df, columns=[col], drop_first=True)

    # Asegurar que tengamos todas las columnas necesarias
    missing_cols = set(feature_names) - set(input_df.columns)
    for col in missing_cols:
        input_df[col] = 0

    # Reordenar columnas
    input_df = input_df[feature_names]

    # Escalar los datos
    input_scaled = scaler.transform(input_df)

    # Predecir
    prediction = model.predict(input_scaled)
    proba = model.predict_proba(input_scaled)[0][1]

    return prediction[0], proba

# Interfaz principal
def main():
    if df is not None:
        X_train_scaled, X_test_scaled, y_train, y_test, scaler, feature_names = preprocess_data(df)

        # Sidebar con opciones
        st.sidebar.header("Opciones")
        app_mode = st.sidebar.selectbox("Seleccione el modo",
                                      ["Información del Modelo", "Evaluación del Modelo", "Predicción Individual", "Predicción por Lote"])

        if app_mode == "Información del Modelo":
            st.header("Modelo Predictivo Optimizado de Alta Precisión")
            st.markdown("""
            **Support Vector Machine (SVC) con kernel polinómico optimizado**

            - **Parámetros:**
                - C: 0.71 (parámetro de regularización)
                - Kernel: polinómico
                - Balanceo de clases: activado
                - Random state: 42
            - **Rendimiento esperado:**
                - Accuracy: ~0.85
                - Recall: ~0.88
                - Precision: ~0.83
                - F1-score: ~0.85
            """)

            st.subheader("Variables Importantes")
            st.markdown("""
            Las características más importantes para la predicción incluyen:
            - Duración de la última llamada
            - Número de contactos en esta campaña
            - Resultado de campañas anteriores
            - Edad del cliente
            - Balance en la cuenta
            """)

            st.subheader("Base de Datos")
            st.write("Vista previa de los datos:")
            st.dataframe(df.head())

        elif app_mode == "Evaluación del Modelo":
            st.header("Evaluación del Modelo")

            if st.button("Entrenar y Evaluar Modelo"):
                with st.spinner("Entrenando modelo..."):
                    model = train_winner_model(X_train_scaled, y_train)
                    y_pred = model.predict(X_test_scaled)
                    y_proba = model.predict_proba(X_test_scaled)[:, 1]

                    # Guardar modelo
                    joblib.dump(model, 'bank_marketing_model.pkl')
                    joblib.dump(scaler, 'scaler.pkl')

                    st.success("Modelo entrenado y guardado correctamente!")

                    # Mostrar métricas en pestañas
                    tab1, tab2, tab3, tab4 = st.tabs(["Métricas", "Matriz de Confusión", "Curva ROC", "Reporte Completo"])

                    with tab1:
                        st.subheader("Métricas Principales")
                        col1, col2, col3, col4 = st.columns(4)
                        col1.metric("Accuracy", f"{accuracy_score(y_test, y_pred):.4f}",
                                  help="Proporción de predicciones correctas")
                        col2.metric("Precision", f"{precision_score(y_test, y_pred):.4f}",
                                  help="Proporción de verdaderos positivos entre todos los positivos predichos")
                        col3.metric("Recall", f"{recall_score(y_test, y_pred):.4f}",
                                  help="Proporción de verdaderos positivos identificados correctamente")
                        col4.metric("F1-score", f"{f1_score(y_test, y_pred):.4f}",
                                  help="Media armónica de precision y recall")

                    with tab2:
                        st.subheader("Matriz de Confusión")
                        cm = confusion_matrix(y_test, y_pred)
                        fig, ax = plt.subplots(figsize=(6, 4))
                        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                                   xticklabels=['No Deposit', 'Deposit'],
                                   yticklabels=['No Deposit', 'Deposit'])
                        ax.set_xlabel('Predicho')
                        ax.set_ylabel('Real')
                        st.pyplot(fig)

                    with tab3:
                        st.subheader("Curva ROC")
                        fpr, tpr, _ = roc_curve(y_test, y_proba)
                        roc_auc = auc(fpr, tpr)
                        fig, ax = plt.subplots(figsize=(8, 6))
                        ax.plot(fpr, tpr, label=f'Área bajo la curva = {roc_auc:.4f}')
                        ax.plot([0, 1], [0, 1], 'k--')
                        ax.set_xlabel('Tasa de Falsos Positivos')
                        ax.set_ylabel('Tasa de Verdaderos Positivos')
                        ax.set_title('Curva ROC')
                        ax.legend(loc="lower right")
                        st.pyplot(fig)

                    with tab4:
                        st.subheader("Reporte de Clasificación")
                        report = classification_report(y_test, y_pred, output_dict=True)
                        report_df = pd.DataFrame(report).transpose()
                        st.dataframe(report_df.style.highlight_max(axis=0))

                        # Opción para descargar el reporte
                        csv = report_df.to_csv(index=True).encode('utf-8')
                        st.download_button(
                            "Descargar Reporte como CSV",
                            csv,
                            "model_report.csv",
                            "text/csv",
                            key='download-csv'
                        )

        elif app_mode == "Predicción Individual":
            st.header("Predicción Individual")
            st.markdown("Complete los datos del cliente para predecir si aceptará el depósito a plazo.")

            # Formulario para entrada de datos
            with st.form("prediction_form"):
                col1, col2 = st.columns(2)

                with col1:
                    age = st.number_input("Edad", min_value=18, max_value=100, value=30)
                    job = st.selectbox("Trabajo", df['job'].unique())
                    marital = st.selectbox("Estado Civil", df['marital'].unique())
                    education = st.selectbox("Educación", df['education'].unique())
                    default = st.selectbox("Tiene crédito en default?", ['no', 'yes'])
                    balance = st.number_input("Balance (€)", value=0)

                with col2:
                    housing = st.selectbox("Tiene crédito hipotecario?", ['no', 'yes'])
                    loan = st.selectbox("Tiene préstamo personal?", ['no', 'yes'])
                    contact = st.selectbox("Medio de contacto", df['contact'].unique())
                    day = st.number_input("Día del mes", min_value=1, max_value=31, value=15)
                    month = st.selectbox("Mes", df['month'].unique())
                    duration = st.number_input("Duración último contacto (seg)", min_value=0, value=180)

                campaign = st.number_input("Número de contactos en esta campaña", min_value=1, value=1)
                pdays = st.number_input("Días desde último contacto", min_value=-1, value=-1)
                previous = st.number_input("Número de contactos antes de esta campaña", min_value=0, value=0)
                poutcome = st.selectbox("Resultado de la campaña anterior", df['poutcome'].unique())

                submitted = st.form_submit_button("Predecir")

                if submitted:
                    input_data = {
                        'age': age,
                        'job': job,
                        'marital': marital,
                        'education': education,
                        'default': default,
                        'balance': balance,
                        'housing': housing,
                        'loan': loan,
                        'contact': contact,
                        'day': day,
                        'month': month,
                        'duration': duration,
                        'campaign': campaign,
                        'pdays': pdays,
                        'previous': previous,
                        'poutcome': poutcome
                    }

                    try:
                        model = joblib.load('bank_marketing_model.pkl')
                        scaler = joblib.load('scaler.pkl')

                        prediction, probability = predict_new_data(model, scaler, input_data, feature_names)

                        st.subheader("Resultado de la Predicción")
                        if prediction == 1:
                            st.success(f"El cliente **SÍ** aceptará el depósito a plazo (probabilidad: {probability:.2%})")
                        else:
                            st.error(f"El cliente **NO** aceptará el depósito a plazo (probabilidad: {1-probability:.2%})")

                        # Mostrar barra de probabilidad
                        st.progress(probability)
                        st.metric("Probabilidad de aceptación", f"{probability:.2%}")

                    except FileNotFoundError:
                        st.warning("Primero debe entrenar el modelo en la pestaña 'Evaluación del Modelo'")

        elif app_mode == "Predicción por Lote":
            st.header("Predicción por Lote")
            st.markdown("Suba un archivo CSV con los datos de los clientes para generar predicciones.")

            uploaded_file = st.file_uploader("Seleccione un archivo CSV", type="csv")

            if uploaded_file is not None:
                try:
                    batch_df = pd.read_csv(uploaded_file)
                    st.write("Vista previa de los datos cargados:", batch_df.head())

                    if st.button("Generar Predicciones"):
                        with st.spinner("Procesando datos..."):
                            # Preprocesar el batch
                            batch_encoded = pd.get_dummies(batch_df,
                                                         columns=batch_df.select_dtypes(include=['object']).columns,
                                                         drop_first=True)

                            # Asegurar que tenemos todas las columnas necesarias
                            missing_cols = set(feature_names) - set(batch_encoded.columns)
                            for col in missing_cols:
                                batch_encoded[col] = 0

                            # Reordenar columnas
                            batch_encoded = batch_encoded[feature_names]

                            # Escalar y predecir
                            scaler = joblib.load('scaler.pkl')
                            model = joblib.load('bank_marketing_model.pkl')

                            batch_scaled = scaler.transform(batch_encoded)
                            predictions = model.predict(batch_scaled)
                            probabilities = model.predict_proba(batch_scaled)[:, 1]

                            # Añadir predicciones al dataframe
                            result_df = batch_df.copy()
                            result_df['Predicción'] = ['Sí' if p == 1 else 'No' for p in predictions]
                            result_df['Probabilidad'] = probabilities

                            st.success("Predicciones completadas!")
                            st.dataframe(result_df)

                            # Opción para descargar resultados
                            csv = result_df.to_csv(index=False).encode('utf-8')
                            st.download_button(
                                "Descargar Resultados",
                                csv,
                                "predicciones_clientes.csv",
                                "text/csv",
                                key='download-batch-csv'
                            )

                except Exception as e:
                    st.error(f"Error al procesar el archivo: {str(e)}")

if __name__ == "__main__":
    main()
'''
with open('app.py', 'w') as f:
    f.write(app_code)
print("Archivo app.py creado")

In [ ]:
!pip install streamlit pyngrok

In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
!pip install streamlit

!streamlit run app.py &>/content/logs.txt &


!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
!./cloudflared tunnel --url http://localhost:8501

2025-06-09T15:05:04Z INF Initiating graceful shutdown due to signal interrupt ...
^C
